In [5]:
import random
import numpy as np
from PIL import Image
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split, WeightedRandomSampler, SubsetRandomSampler
from torchvision.transforms import Compose, ToPILImage, ToTensor, Normalize, RandomHorizontalFlip, Resize
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline


import data_generation
print(dir(data_generation))


['EqualGroupSize', 'PaperDataGenerator', 'UnequalGroupSize', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'dgen', 'logger', 'logging', 'np', 'papers_dataset', 'stn_calculator']
